In [53]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['text.usetex'] = True
plt.rcParams['font.size'] = 12
plt.figsize = (10, 20)

# Helper Function
def display_image(image, title='Press Escape to Quit'):
    cv.imshow(title, image)
    # Press escape to close the image
    if cv.waitKey(0) & 0xff == 27:
        cv.destroyAllWindows()

1. Use different thresholding algorithms on the following image to generate and save
the binary image after thresholding.
![](input/thresholding_input.png)

In [54]:
threshold_input = cv.imread('input/thresholding_input.png', cv.IMREAD_GRAYSCALE)
#display_image(threshold_input)

In [55]:
"""
Non Adaptive Thresholding
"""

def manual_threshold(image, threshold, *, display=False, path="output/thresholding/manual_thresholding.png"):
    modified_image = (image > threshold) * 255
    if display:
        display_image(modified_image)
    cv.imwrite(path, modified_image)
    return modified_image

def mean_threshold(image, *, display=False, path="output/thresholding/mean_thresholding.png"):
    modified_image = (image > np.mean(image)) * 255
    if display:
        display_image(modified_image)
    cv.imwrite(path, modified_image)
    return modified_image

def median_threshold(image, *, display=False, path="output/thresholding/median_thresholding.png"):
    modified_image = (image > np.median(image)) * 255
    if display:
        display_image(modified_image)
    cv.imwrite(path, modified_image)
    return modified_image

def otsu_threshold(image, *, display=False, path="output/thresholding/otsu_thresholding.png"):
    hist, bin_edges = np.histogram(image, bins=256)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    
    weight1 = np.cumsum(hist)
    weight2 = np.cumsum(hist[::-1])[::-1]
    
    mean1 = np.cumsum(hist * bin_centers) / weight1
    mean2 = (np.cumsum((hist * bin_centers)[::-1]) / weight2[::-1])[::-1]
    
    variance = weight1[:-1] * weight2[1:] * (mean1[:-1] - mean2[1:]) ** 2
    idx = np.argmax(variance)
    threshold = bin_centers[idx]
    
    modified_image = (image > threshold) * 255
    if display:
        display_image(modified_image)
    cv.imwrite(path, modified_image)
    return modified_image

In [56]:
"""
Adaptive Thresholding
"""

def adaptive_mean_thresholding(image, shift=0, block_size=7, *, display=False, path="output/thresholding/adaptive_mean_thresholding.png"):
    height, width = image.shape
    modified_image = np.zeros_like(image)
    half_block = block_size // 2
    
    for y in range(height):
        for x in range(width):
            
            y1 = max(0, y-half_block)
            y2 = min(height, y+half_block)
            x1 = max(0, x-half_block)
            x2 = min(width, x+half_block)
            
            neighborhood = image[y1:y2, x1:x2]
            threshold = np.mean(neighborhood) - shift
            modified_image[y, x] = 255 if image[y, x] > threshold else 0
            
    if display:
        display_image(modified_image)
        
    cv.imwrite(path, modified_image)
    return modified_image
    
def adaptive_gaussian_thresholding(image, shift=0, block_size=7, *, display=False, path="output/thresholding/adaptive_gaussian_thresholding.png"):
    height, width = image.shape
    modified_image = np.zeros_like(image)
    half_block = block_size // 2
    
    x_values = np.arange(-half_block, half_block+1)
    # Values I set: μ=0, σ=half_block/3
    μ = 0
    σ = half_block/3
    gaussian_kernel = np.exp(- (x_values - μ) ** 2 / (2 * σ ** 2))
    standard_gaussian_kernel = gaussian_kernel / np.sum(gaussian_kernel)
    
    for y in range(height):
        for x in range(width):
            y1 = max(0, y-half_block)
            y2 = min(height, y+half_block)
            x1 = max(0, x-half_block)
            x2 = min(width, x+half_block)
            
            neighborhood = image[y1:y2, x1:x2]
            
            # Pad the neighborhood with zeros
            pad_y = half_block - (y - y1)
            pad_x = half_block - (x - x1)
            padded_neighborhood = np.pad(neighborhood,
                                         ((pad_y, block_size - neighborhood.shape[0] - pad_y),
                                          (pad_x, block_size - neighborhood.shape[1] - pad_x)), 
                                            mode='edge'
                                        )
            
            weighted_sum = np.sum(padded_neighborhood * np.outer(standard_gaussian_kernel, standard_gaussian_kernel))
            threshold = weighted_sum - shift
            modified_image[y, x] = 255 if image[y, x] > threshold else 0
            
    if display:
        display_image(modified_image)
        
    cv.imwrite(path, modified_image)
    return modified_image

In [57]:
manual_threshold(threshold_input, 127)
mean_threshold(threshold_input)
median_threshold(threshold_input)
otsu_threshold(threshold_input)
print("Done")


Done


In [58]:
adaptive_mean_thresholding(threshold_input, 4)
adaptive_gaussian_thresholding(threshold_input, 4)
print("Done")

Done


2.Draw a fence of an arbitrary shape by hand (!) on a clean paper with white background
and printed lines on it as explained in the class. Take a photo of this image. Write a
program\
a) to read the photographed image and translate it to a gray scale image of size say,
300 x 300 of this image.\
b) Use global thresholding algorithm to clean image to remove all lines on it.\
c) Save original and binary image by name fence_original.jpg and
fence_threshold.jpg.\
d) Write a program to draw a filled circle of radius 5 randomly in this image
(fence_threshold.jpg). Generate 50 such images. Store these images into two
folders as explained in the class.

In [59]:
fence_image = cv.imread('input/fence.jpeg', cv.IMREAD_GRAYSCALE)
fence_image = cv.resize(fence_image, (300, 300))
thresholded_image = adaptive_gaussian_thresholding(fence_image, 20, path="output/fence/adaptive_gaussian_thresholding.png")

In [62]:
def draw_circle(image, radius, center_x, center_y):
    image_copy = image.copy()
    rr, cc = np.mgrid[:image_copy.shape[0], :image_copy.shape[1]]
    circle = (rr - center_y)**2 + (cc - center_x)**2 <= radius**2
    image_copy[circle] = 0
    return image_copy

cv.imwrite("output/fence/circles/circle_0.png", thresholded_image)

num_images = 50
for i in range(num_images):
    random_radius = np.random.randint(5, 20)
    random_center_x = np.random.randint(random_radius + 1, thresholded_image.shape[1] - random_radius - 1)
    random_center_y = np.random.randint(random_radius + 1, thresholded_image.shape[0] - random_radius - 1)
    modified_image = draw_circle(thresholded_image, random_radius, random_center_x, random_center_y)
    cv.imwrite(f"output/fence/circles/circle_{i + 1}.png", modified_image)
print("Done")

Done


3. Write a histogram equalization code to improve the contrast of the following grayscale image.\
A)  Show and save the input and output (after histogram equalization) grayscale images.\
B)  Show and save the intensity histogram of the input and output image.\
C)  Show and save the differential probability histogram of the input and output image.\
D)  Superimpose the cumulative probability histogram on c; the differential probability histogram of the input and output image.\
E)  Obtain and store the mean intensity of the input and output image

In [77]:
brain_image = cv.imread("input/brain.png", cv.IMREAD_GRAYSCALE)

In [90]:
def histogram(image, name=""):
    histogram = np.zeros(256)
    for pixel_intensity in image.flatten():
        histogram[pixel_intensity] += 1

    if name:
        plt.figure()
        plt.bar(np.arange(256), histogram)
        plt.xlabel(r'Pixel Intensity $\longrightarrow$')
        plt.ylabel(r'Frequency $\longrightarrow$')
        plt.title(f'Histogram of {name} image')
        plt.savefig(f'output/brain/{name}_histogram.png')
        plt.close()
    
    return histogram

def differential_histogram(image, name=""):
    hist = histogram(image)
    differential_histogram = hist / image.size

    if name:
        plt.figure()
        plt.bar(np.arange(256), differential_histogram)
        plt.xlabel(r'Pixel Intensity $\longrightarrow$')
        plt.ylabel(r'Probability $\longrightarrow$')
        plt.title(f'Differential Histogram of {name} image')
        plt.savefig(f'output/brain/{name}_differential_histogram.png')
        plt.close()
    
    return differential_histogram

def cumulative_histogram(image, name=""):
    diff_hist = differential_histogram(image)
    cumulative_histogram = np.cumsum(diff_hist)

    if name:
        plt.figure()
        plt.bar(np.arange(256), cumulative_histogram)
        plt.xlabel(r'Pixel Intensity $\longrightarrow$')
        plt.ylabel(r'Probability $\longrightarrow$')
        plt.title(f'Cumulative Histogram of {name} image')
        plt.savefig(f'output/brain/{name}_cumulative_histogram.png')
        plt.close()
    
    return cumulative_histogram

def histogram_equalization(image, name=""):
    cum_hist = cumulative_histogram(image)
    equalized_values = np.interp(image.flatten(), np.arange(256), cum_hist * 255)
    modified_image = equalized_values.reshape(image.shape)

    if name:
        cv.imwrite(f'output/brain/{name}_histogram_equalized.png', modified_image)
    
    return modified_image.astype(np.uint8)

In [91]:
cv.imwrite('output/brain/input_image.png', brain_image)

equalized_image = histogram_equalization(brain_image)
cv.imwrite('output/brain/output_image.png', equalized_image)

histogram(brain_image, name="input")
histogram(equalized_image, name="output")

diff_hist_in = differential_histogram(brain_image, name="input")
diff_hist_out = differential_histogram(equalized_image, name="output")

cum_hist_in = cumulative_histogram(brain_image)
cum_hist_out = cumulative_histogram(equalized_image)

In [92]:
plt.figure()
plt.bar(np.arange(256), diff_hist_in, label="Differential")
plt.plot(np.arange(256), cum_hist_in, color='red', label="Cumulative")
plt.xlabel(r'Pixel Intensity $\longrightarrow$')
plt.ylabel(r'Probability $\longrightarrow$')
plt.title('Input Image: Combined Histograms')
plt.legend()
plt.savefig('output/brain/input_combined_histograms.png')
plt.close()

plt.figure()
plt.bar(np.arange(256), diff_hist_out, label="Differential")
plt.plot(np.arange(256), cum_hist_out, color='red', label="Cumulative")
plt.xlabel(r'Pixel Intensity $\longrightarrow$')
plt.ylabel(r'Probability $\longrightarrow$')
plt.title('Output Image: Combined Histograms')
plt.legend()
plt.savefig('output/brain/output_combined_histograms.png')
plt.close()

mean_intensity_input = np.mean(brain_image)
mean_intensity_output = np.mean(equalized_image)
print(f"Mean Intensity of Input Image: {mean_intensity_input}")
print(f"Mean Intensity of Output Image: {mean_intensity_output}")

Mean Intensity of Input Image: 76.72112436809559
Mean Intensity of Output Image: 129.4994793514872
